In [40]:
import numpy as np
import pandas as pd
import sklearn


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [2]:
data = pd.read_csv("train.csv")

In [3]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
data.shape

(7613, 5)

In [5]:
data.keyword.value_counts()

fatalities               45
armageddon               42
deluge                   42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [104]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus=[]
n  =0
for idex, row in data.iterrows():
  ligne = row['text'].lower()
  ligne = ligne.split()
  ps = PorterStemmer()
  #ligne = [ps.stem(word) for word in ligne ]
  all_stopwords = stopwords.words('english')
  review = [ps.stem(word) for word in ligne if not word in set(all_stopwords)]


  #ligne = row['text'].split(' ')
  n += len(ligne)
  review = ' '.join(ligne)
  corpus.append(review)
print("La longeur moyenne des tweets en nombre de mots : ", n/data.shape[0])


[nltk_data] Downloading package stopwords to /home/m1tal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


La longeur moyenne des tweets en nombre de mots :  14.903585971364771


In [105]:
data.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [106]:
target0 =  data[data['target'] == 0]
target1 =  data[data['target'] == 1]

In [107]:
print(target0.shape)
print(target1.shape)

(4342, 5)
(3271, 5)


In [108]:
data.isna().sum().sort_values(ascending= False) / data.shape[0]

location    0.332720
keyword     0.008013
id          0.000000
text        0.000000
target      0.000000
dtype: float64

In [109]:
n  =0
for idex, row in target0.iterrows():
  ligne = row['text'].split(' ')
  n += len(ligne)
  #print(ligne)
print(" TARGET 0 : La longeur moyenne des tweets en nombre de mots : ", n/data.shape[0])


 TARGET 0 : La longeur moyenne des tweets en nombre de mots :  8.397609352423487


In [110]:
n  =0
for idex, row in target1.iterrows():
  ligne = row['text'].split(' ')
  n += len(ligne)
  #print(ligne)
print("TARGET 1 : La longeur moyenne des tweets en nombre de mots : ", n/data.shape[0])


TARGET 1 : La longeur moyenne des tweets en nombre de mots :  6.5313279915933276


In [111]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(data.text)
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
vectorizer = TfidfVectorizer()

#XX = vectorizer2.fit_transform(data.text)
#X = X.toarray()
#XX = XX.toarray()



XX = vectorizer.fit_transform(corpus).toarray()


In [119]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XX, data.target, test_size=0.2, random_state=0)



In [113]:
print(X_train.shape)
print(X_test.shape)

(6090, 21637)
(1523, 21637)


In [114]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(y_test[:15])
print(y_pred[:15])

from sklearn.naive_bayes import MultinomialNB
gnb = MultinomialNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(y_test[:15])
print(y_pred[:15])

from sklearn.svm import LinearSVC
gnb = LinearSVC()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(y_test[:15])
print(y_pred[:15])


311     0
4970    0
527     0
6362    0
800     0
3074    0
1833    0
2272    0
1234    1
1326    0
5516    0
5056    1
3447    0
2211    0
3286    1
Name: target, dtype: int64
[1 1 1 0 1 0 0 0 1 1 1 1 0 0 1]
311     0
4970    0
527     0
6362    0
800     0
3074    0
1833    0
2272    0
1234    1
1326    0
5516    0
5056    1
3447    0
2211    0
3286    1
Name: target, dtype: int64
[0 0 1 0 0 0 0 0 1 0 0 0 0 0 0]
311     0
4970    0
527     0
6362    0
800     0
3074    0
1833    0
2272    0
1234    1
1326    0
5516    0
5056    1
3447    0
2211    0
3286    1
Name: target, dtype: int64
[0 1 0 0 0 0 0 0 1 0 0 1 0 0 1]


In [116]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred, labels= [0, 1]))

              precision    recall  f1-score   support

     class 0       0.81      0.89      0.85       886
     class 1       0.82      0.70      0.76       637

    accuracy                           0.81      1523
   macro avg       0.81      0.80      0.80      1523
weighted avg       0.81      0.81      0.81      1523

[[788  98]
 [189 448]]
